# Hidden Markov Models (HMM) — An approachable notebook

**Target audience:** Intermediate students in Machine Learning familiar with Python, NumPy, and scikit-learn API patterns.

**Goal:** teach HMM concepts, implement and fit HMMs using the `hmmlearn` library (sklearn-like API), explore a dataset (synthetic but realistic), evaluate models, perform decoding (Viterbi), and integrate HMMs into common model-selection workflows.

**Notebook style:** explanatory text + runnable examples + visualization and metrics.

## 1 — Setup: libraries and environment

This notebook uses `hmmlearn` (a commonly used HMM package with a scikit-learn-like API). If you don't have it installed, run `pip install hmmlearn` in the environment where you run the notebook.

We'll also use standard packages: `numpy`, `pandas`, `matplotlib`, and `scikit-learn` for metrics and utilities.

In [ ]:
"""
Standard imports
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# HMM package (sklearn-like)
try:
    from hmmlearn import hmm
except Exception as e:
    raise ImportError("hmmlearn is required. Install with: pip install hmmlearn") from e

# Reproducibility
RND = 42
np.random.seed(RND)

print('numpy:', np.__version__)
print('pandas:', pd.__version__)
print('hmmlearn:', getattr(hmm, '__version__', 'unknown'))

## 2 — Quick theory recap (brief)

- **Hidden Markov Model (HMM)**: a generative probabilistic model for sequential data. The model contains:
  - A discrete hidden state sequence \(S_t\) (e.g., weather: {sunny, rainy}).
  - Observations \(X_t\) generated from an emission distribution conditioned on the current hidden state.
  - A transition matrix \(A\) that governs \(P(S_t | S_{t-1})\).
  - An initial state distribution \(\pi\).

Common tasks:
1. **Likelihood estimation** (fit parameters given observed sequences).
2. **Decoding** (find the most likely hidden state sequence given observations, Viterbi).
3. **Scoring / model selection** (log-likelihood, AIC/BIC).

This notebook uses **Gaussian emissions** (continuous observations) for clarity and visualization.

## 3 — Construct a synthetic dataset (known ground truth)

We build a 3-state HMM with 2D Gaussian emissions. We'll sample many short sequences to simulate realistic time-series data. We keep the ground-truth transition and emission parameters so we can evaluate how well the model recovers them.

In [ ]:
"""
Ground-truth HMM parameters (3 hidden states)
"""

n_states = 3
n_dim = 2  # 2D continuous observations so we can plot them easily

# Transition matrix (rows sum to 1)
trans_mat = np.array([
    [0.7, 0.2, 0.1],
    [0.1, 0.8, 0.1],
    [0.2, 0.3, 0.5]
])

# Start probabilities
start_prob = np.array([0.6, 0.3, 0.1])

# Emission parameters: Gaussian means and covariances for each state
means = np.array([[0.0, 0.0],
                  [4.0, 2.0],
                  [-3.0, 4.0]])

covars = np.array([[[0.5, 0.05],[0.05, 0.4]],
                   [[0.7, -0.02],[-0.02, 0.6]],
                   [[0.6, 0.0],[0.0, 0.5]]])

# Small function to sample sequences from this HMM
def sample_hmm(trans_mat, start_prob, means, covars, n_sequences=200, seq_len=50, random_state=None):
    rng = np.random.RandomState(random_state)
    sequences = []      # list of arrays, each sequence
    lengths = []        # lengths of each sequence
    state_seqs = []     # true hidden states for each sequence (for evaluation)
    for _ in range(n_sequences):
        states = []
        obs = []
        # initial state
        s = rng.choice(len(start_prob), p=start_prob)
        for t in range(seq_len):
            states.append(s)
            obs.append(rng.multivariate_normal(means[s], covars[s]))
            s = rng.choice(len(start_prob), p=trans_mat[s])
        sequences.append(np.vstack(obs))
        lengths.append(seq_len)
        state_seqs.append(np.array(states))
    return sequences, lengths, state_seqs

# Create dataset
n_sequences = 300
seq_len = 40
sequences, lengths, state_seqs = sample_hmm(trans_mat, start_prob, means, covars,
                                            n_sequences=n_sequences, seq_len=seq_len, random_state=RND)

# Concatenate into a single long observation array (hmmlearn likes this for multiple sequences)
X = np.vstack(sequences)
lengths = lengths  # list of lengths
y_true = np.hstack(state_seqs)  # flattened true states for every timestep
print('Total timesteps:', X.shape[0])
print('Number of sequences:', len(sequences))

### 3.1 — Dive into the dataset

Let's look at the observations, a scatter plot colored by true (hidden) state, and some quick statistics (mean, covariance per state).

In [ ]:
"""
Basic stats and visualization
"""

import matplotlib.patches as mpatches

# Scatter plot of observations colored by true hidden state (using flattened arrays)
fig, ax = plt.subplots(figsize=(8,6))
colors = ['C0','C1','C2']
for s in range(n_states):
    mask = (y_true == s)
    ax.scatter(X[mask,0], X[mask,1], s=8, label=f'state {s}', alpha=0.6)
ax.set_title('Observations colored by true hidden state (synthetic)')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.legend()
plt.show()

# Compute empirical means / covariances for each known state (sanity check)
emp_stats = []
for s in range(n_states):
    xs = X[y_true==s]
    emp_stats.append((xs.mean(axis=0), np.cov(xs.T)))
    print(f"State {s}: mean = {emp_stats[-1][0]}, cov shape = {emp_stats[-1][1].shape}")

## 4 — Fit an HMM (Gaussian emissions) using `hmmlearn`

`hmmlearn`'s `GaussianHMM` provides a fit / predict API similar to scikit-learn models. We'll:

1. Fit a model with the correct number of states.
2. Decode the most likely hidden state sequence (Viterbi).
3. Compare learnt parameters with ground-truth.

Notes: `hmmlearn` expects concatenated observations and a list of sequence lengths for fitting multiple sequences.

In [ ]:
"""
Fit a GaussianHMM
"""

model = hmm.GaussianHMM(n_components=n_states, covariance_type='full', n_iter=200, random_state=RND, verbose=False)
model = model.fit(X, lengths)

print('Estimated startprob:', model.startprob_)
print('Estimated transmat:\n', model.transmat_)
print('Estimated means:\n', model.means_)
print('Estimated covariances shapes:', [c.shape for c in model.covars_])

### 4.1 — Decoding (Viterbi) and evaluation

We'll use Viterbi to get the most likely hidden state sequence for the entire concatenated observation series and then compute simple metrics against the ground truth. Keep in mind: HMM hidden states are *unlabeled* — the numbering is arbitrary. We'll find the best mapping from predicted states to true states using the Hungarian algorithm.

In [ ]:
"""
Viterbi decode (predict hidden states)
"""

# Predict hidden states for the whole dataset
y_pred = model.predict(X, lengths)

# Because states are unlabeled, compute mapping via Hungarian matching on confusion matrix
from scipy.optimize import linear_sum_assignment

cm = confusion_matrix(y_true, y_pred)
# We want to maximize correct matches -> minimize negative of matrix for Hungarian algorithm
row_ind, col_ind = linear_sum_assignment(-cm)
mapping = dict(zip(col_ind, row_ind))

# Map predicted states to true labels where possible
y_pred_mapped = np.vectorize(lambda s: mapping.get(s, s))(y_pred)

acc = accuracy_score(y_true, y_pred_mapped)
print(f'Accuracy after optimal mapping: {acc:.4f}')

# Show confusion matrix (mapped)
cm_mapped = confusion_matrix(y_true, y_pred_mapped)
print('Confusion matrix (true rows, predicted columns after mapping):\n', cm_mapped)

### 4.2 — Visualize decoding on an example sequence

Plot true vs decoded states for one example sequence (time series).

In [ ]:
"""
Plot one sequence's observations and true vs predicted states
"""

# Choose a sequence to visualize
seq_idx = 5
obs = sequences[seq_idx]
true_states = state_seqs[seq_idx]

# Predict for this specific sequence using model.predict on the slice
start = seq_idx * seq_len
end = start + seq_len
pred_seq = y_pred_mapped[start:end]

fig, ax = plt.subplots(2,1, figsize=(10,5), sharex=True)
ax[0].plot(obs[:,0], label='obs dim 0')
ax[0].plot(obs[:,1], label='obs dim 1')
ax[0].set_title('Observations (sequence {})'.format(seq_idx))
ax[0].legend()

ax[1].step(range(seq_len), true_states, where='mid', label='true state')
ax[1].step(range(seq_len), pred_seq, where='mid', label='predicted state (mapped)')
ax[1].set_title('True vs Predicted hidden states (sequence {})'.format(seq_idx))
ax[1].legend()
plt.tight_layout()
plt.show()

### 4.3 — Compare learned parameters with ground truth

Parameter recovery is not guaranteed (identifiability issues, limited data), but we can eyeball how similar the estimated means and transition matrix are to the true ones.

In [ ]:
print('Ground-truth means:\n', means)
print('\nEstimated means (order corresponds to model.states):\n', model.means_)

print('\nGround-truth transmat:\n', trans_mat)
print('\nEstimated transmat:\n', model.transmat_)

## 5 — Model selection: how many hidden states?

Common model selection strategies:
- Compare models with different `n_components` using held-out log-likelihood, AIC, or BIC.

We'll compute BIC and AIC for a range of components and show selection. Note: exact formulas depend on the number of free parameters; for HMMs, compute carefully.

In [ ]:
"""
Helper to compute number of free parameters for GaussianHMM (approximate)
"""

def n_hmm_params(n_components, n_features, covariance_type='full'):
    # start probabilities: n_components - 1 free parameters (sum-to-one)
    startp = n_components - 1
    # transition matrix: n_components*(n_components-1) free params (each row sums to 1)
    trans = n_components * (n_components - 1)
    # Emission parameters: means (n_components * n_features) and covariances
    if covariance_type == 'full':
        cov_params = n_components * (n_features * (n_features + 1) / 2)
    elif covariance_type == 'diag':
        cov_params = n_components * n_features
    else:
        cov_params = n_components * n_features  # fallback
    return int(startp + trans + n_components * n_features + cov_params)

# Fit models with different component counts and compute AIC/BIC
results = []
max_components = 6
for k in range(1, max_components+1):
    m = hmm.GaussianHMM(n_components=k, covariance_type='full', n_iter=200, random_state=RND)
    m.fit(X, lengths)
    ll = m.score(X, lengths)  # total log-likelihood for all data
    p = n_hmm_params(k, n_dim, covariance_type='full')
    n_obs = X.shape[0]
    aic = -2 * ll + 2 * p
    bic = -2 * ll + p * np.log(n_obs)
    results.append({'n_components': k, 'log_likelihood': ll, 'n_params': p, 'AIC': aic, 'BIC': bic})

import pandas as pd

_df_results = pd.DataFrame(results).set_index('n_components')
_df_results

In [ ]:
_df_results[['AIC','BIC']].plot(marker='o', figsize=(8,4))
plt.title('AIC & BIC vs number of HMM components')
plt.xlabel('n_components')
plt.ylabel('Information Criterion')
plt.show()

## 6 — Practical tips and gotchas

- **Label switching**: HMM states have no intrinsic order. Use matching strategies (Hungarian algorithm) when you have ground truth.
- **Initialization matters**: `hmmlearn` uses random starts; run multiple initializations or provide sensible `means_init`/`transmat_init`.
- **Sequence lengths**: if you have sequences of varying lengths, pass the `lengths` list to `fit`/`score`.
- **Model selection**: AIC/BIC help but are not foolproof. Consider held-out likelihood or predictive performance on a downstream task.
- **Scaling**: standardize features when needed (makes sense for Gaussian emissions).

## 7 — Extensions & exercises (for students)

1. Try discrete HMMs (`MultinomialHMM`) on discrete sequences (e.g., text tokens) and compare training behavior.
2. Implement multiple random restarts and show how initialization affects final log-likelihood.
3. Use a real dataset: e.g., sensor data, human activity sequences, or simplified speech data. Preprocess, then fit HMMs and interpret states.
4. Create a supervised evaluation: train HMMs with different numbers of states and evaluate on held-out sequences using Viterbi-decoded labels.

---

That's the end of the notebook. Feel free to ask for expansions: adding MultinomialHMM examples, integrating with scikit-learn pipelines, or porting examples to a real dataset.